In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install h2o
!pip install wandb

In [ ]:
# Use H2O4GPU
# export some env variable
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64/:$CUDA_HOME/lib/:$CUDA_HOME/extras/CUPTI/lib64

#install python packges
!sudo apt-get install libopenblas-dev pbzip2
!sudo apt-get -y install libcurl4-openssl-dev libssl-dev libxml2-dev

# 要重啟kernel
# uninstall sklearn and install python packges
!pip uninstall scikit-learn -y
!pip install -U --force-reinstall ipykernel
!pip install parallel
!pip install --force-reinstall -I h2o4gpu

In [ ]:
# 即時監控colab資源
import wandb
wandb.init()

In [2]:
# import packages
import os
import numpy as np
import pandas as pd

# H2O
import h2o
from h2o.automl import H2OAutoML

import catboost
import lightgbm
import xgboost

import h2o4gpu as sklearn
from sklearn.preprocessing import LabelEncoder
from joblib import load, dump

C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# connect with Google Cloud
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/colab/TPS May"
# path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - May 2021'
os.chdir(path)

In [4]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
# Initialize and Connect to H2O
h2o.init()

hof_train = h2o.H2OFrame(df_train.copy())
hof_test = h2o.H2OFrame(df_test.copy())

feature_col = [i for i in df_train.columns if i not in ['id', 'target']]
target_col = 'target'

# for binary or multiclass
hof_train[target_col] = hof_train[target_col].asfactor()

# Warnings : Xgboost is not supported on windows
h2o.estimators.xgboost.H2OXGBoostEstimator.available()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu11.43+55-CA (build 11.0.9.1+1-LTS, mixed mode)
  Starting server from C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Chen\AppData\Local\Temp\tmpz8flxsvt
  JVM stdout: C:\Users\Chen\AppData\Local\Temp\tmpz8flxsvt\h2o_Chen_started_from_python.out
  JVM stderr: C:\Users\Chen\AppData\Local\Temp\tmpz8flxsvt\h2o_Chen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Taipei
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_Chen_8aqldl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.955 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Cannot build an XGBoost model - no backend found.


False

In [6]:
# construct model
PROJECT_NAME = 'h2o-20210522'
automl = H2OAutoML(project_name = PROJECT_NAME,
                   seed=42,
                   max_runtime_secs=6*60*60,
                   nfolds = 5,
                   balance_classes = True,
                   # include_algos = ['DRF', 'GLM', 'XGBoost', 'GBM', "DeepLearning", 'StackedEnsemble'],
                   exclude_algos = ["DeepLearning"],
                   )

In [ ]:
# training model
automl.train(x=feature_col, 
             y=target_col, 
             training_frame=hof_train)

In [ ]:
# Save all model
k = len(automl.leaderboard)
for i in range(k):
   aml = h2o.get_model(automl.leaderboard[i, 0])
   h2o.save_model(model=aml, path=PROJECT_NAME + '/ALL_MODEL', force=True)

In [ ]:
# save best model
best_model_path = h2o.save_model(model=automl.leader, path=PROJECT_NAME, force=True)

In [ ]:
# load best model
best_model = h2o.load_model(best_model_path)

In [ ]:
# predict probability
result = best_model.predict(hof_test).as_data_frame()
print(result.head())

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result.iloc[:, 1:]
sub.to_csv(f'{PROJECT_NAME}.csv', index=False)

In [ ]:
# detail
lb = automl.leaderboard 
print(lb.as_data_frame())

In [ ]:
# 關閉H2O
h2o.shutdown()